# Benchmarks

Against some alternatives.

## Data

UCI Credit card dataset with 30k rows and 23 features.

In [50]:
import pandas as pd
from skorecard.datasets import load_credit_card
from sklearn.model_selection import train_test_split

data = load_credit_card(as_frame=True)
print(f"data shape: {data.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['y'], axis=1),
    data['y'], 
    test_size=0.25, 
    random_state=42
)

data shape: (30000, 24)


## Experiment setup

In [51]:
from sklearn.metrics import roc_auc_score

def report_auc(clf, X_train, y_train, X_test, y_test):
    proba_train = clf.predict_proba(X_train)[:,1]
    proba_test = clf.predict_proba(X_test)[:,1]

    auc_train = round(roc_auc_score(y_train, proba_train),4) 
    auc_test = round(roc_auc_score(y_test, proba_test),4)

    return auc_train, auc_test


In [52]:
from memo import memlist, time_taken

data = []

@memlist(data=data)
@time_taken()
def fit_eval_record(clf, name):
    
    clf.fit(X_train, y_train)
    auc_train, auc_test = report_auc(clf, X_train, y_train, X_test, y_test)

    return {'auc_train': auc_train, 'auc_test': auc_test}

## Baseline

In [63]:
from skorecard import Skorecard

scorecard = Skorecard()
fit_eval_record(scorecard, name="skorecard.Scorecard")

# old 0.7179
# WOEEn 0.766


/Users/ue86yw/Developer/miniconda3/envs/py38/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


{'auc_train': 0.7727, 'auc_test': 0.766, 'time_taken': 7.29}

In [62]:
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LogisticRegression
# from skorecard.preprocessing import WoeEncoder
# from skorecard.bucketers import DecisionTreeBucketer, OptimalBucketer
# from category_encoders.woe import WOEEncoder

# pipe = make_pipeline(
#     DecisionTreeBucketer(),
#     OptimalBucketer(),
#     #WoeEncoder(),
#     WOEEncoder(cols=X_train.columns),
#     LogisticRegression(solver="lbfgs", max_iter=400)
# )

# fit_eval_record(pipe, name="pipeline")

# # .7166 with skorecard woe in 3.7s
# # 0.758 with no WOE in 3.9s
# # 0.7661 with WOE on all cols.

/Users/ue86yw/Developer/miniconda3/envs/py38/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


{'auc_train': 0.7726, 'auc_test': 0.7661, 'time_taken': 3.97}

## Optbinning

http://gnpalencia.org/optbinning/

In [ ]:
from optbinning import BinningProcess
from optbinning import Scorecard
from sklearn.linear_model import LogisticRegression
import pandas as pd

selection_criteria = {
    "iv": {"min": 0.02, "max": 1},
    "quality_score": {"min": 0.01}
}
binning_process = BinningProcess(variable_names = list(X_train.columns), selection_criteria=selection_criteria)

estimator = LogisticRegression(solver="lbfgs")

opt_scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 300, "max": 850})


fit_eval_record(opt_scorecard, name="optbinning.Scorecard")

## Basic LR

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression(random_state=42, solver="lbfgs")
)

fit_eval_record(pipe, name="sklearn.LogisticRegression")

## LightGBM model

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(random_state=42, max_depth=10, learning_rate=0.01)

fit_eval_record(clf, name="LightGBM")

## Results

In [ ]:
pd.DataFrame(data).sort_values('auc_test', ascending=False)